# MGP: eliminación de variables

Curso: [Aprendizaje Automatizado](http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado). Profesor: [Gibran Fuentes Pineda](https://turing.iimas.unam.mx/~gibranfp/). Ayudantes: [Bere](https://turing.iimas.unam.mx/~bereml/) y [Ricardo](https://turing.iimas.unam.mx/~ricardoml/) Montalvo Lezama.

---
---

En esta libreta realizaremos consultas sencillas sobre dos modelos. Utilizaremos [pgmpy](https://github.com/pgmpy/pgmpy), una biblioteca libre python para implementar modelos gráficos probabilísticos.

In [1]:
try:
  import google.colab
  !pip install pgmpy
except:
  pass

from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

     |████████████████████████████████| 1.9 MB 4.8 MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Ejemplo 1

<img src="https://raw.githubusercontent.com/gibranfp/CursoAprendizajeAutomatizado/master/notebooks/fig/ev-ej1.jpg">

### 1. Construcción del modelo

In [2]:
# definición de modelo global
ejemplo1_modelo = BayesianModel([('A', 'B'), 
                              ('B', 'C'),
                              ('B', 'D')])

/usr/local/lib/python3.7/dist-packages/pgmpy/models/BayesianModel.py:10: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  FutureWarning,


In [3]:
# definición de distribuciones condicionales
dpc_a = TabularCPD(variable='A', variable_card=2,
                     values=[[0.8], [0.2]])
dpc_b = TabularCPD(variable='B', variable_card=2,
                     values=[[0.9, 0.7],
                             [0.1, 0.3]],
                     evidence=['A'],
                     evidence_card=[2])
dpc_c = TabularCPD(variable='C', variable_card=2,
                     values=[[0.9, 0.5],
                             [0.1, 0.5]],
                     evidence=['B'],
                     evidence_card=[2])
dpc_d = TabularCPD(variable='D', variable_card=2,
                     values=[[0.7, 0.4],
                             [0.3, 0.6]],
                     evidence=['B'],
                     evidence_card=[2])

In [4]:
# asociación de distribuaciones a modelo
ejemplo1_modelo.add_cpds(dpc_a, dpc_b, dpc_c, dpc_d)

# verificación del modelo
ejemplo1_modelo.check_model()

True

### 2. Inferencia

In [5]:
# instanciando objecto de eliminación de variables
ejemplo1_inf = VariableElimination(ejemplo1_modelo)

# haciendo consulta
consulta_b_c = ejemplo1_inf.query(variables=['B'], evidence={'C': 0})

print(consulta_b_c)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

+------+----------+
| B    |   phi(B) |
+======+==========+
| B(0) |   0.9171 |
+------+----------+
| B(1) |   0.0829 |
+------+----------+


### Ejemplo 2

<img src="https://raw.githubusercontent.com/gibranfp/CursoAprendizajeAutomatizado/master/notebooks/fig/ev-ej2.jpg">

### 1. Construcción del modelo

In [6]:
empleo_modelo = BayesianModel([('Experiencia', 'Entrevista'), 
                              ('Promedio', 'Entrevista'),
                              ('Promedio', 'Admision'),
                              ('Entrevista', 'Oferta')])

/usr/local/lib/python3.7/dist-packages/pgmpy/models/BayesianModel.py:10: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  FutureWarning,


In [7]:
# definición de distribuciones condicionales
dpc_exp = TabularCPD(variable='Experiencia', variable_card=2,
                     values=[[0.7], [0.3]])
dpc_pro = TabularCPD(variable='Promedio', variable_card=2,
                     values=[[0.2], [0.8]])
dpc_ent = TabularCPD(variable='Entrevista', variable_card=2,
                     values=[[0.9, 0.6, 0.3, 0.1],
                             [0.1, 0.4, 0.7, 0.9]],
                     evidence=['Experiencia', 'Promedio'],
                     evidence_card=[2, 2])
dpc_adm = TabularCPD(variable='Admision', variable_card=2,
                     values=[[0.9, 0.2],
                             [0.1, 0.8]],
                     evidence=['Promedio'],
                     evidence_card=[2])
dpc_ofe = TabularCPD(variable='Oferta', variable_card=2,
                     values=[[0.9, 0.2],
                             [0.1, 0.8]],
                     evidence=['Entrevista'],
                     evidence_card=[2])

In [8]:
# asociación de distribuciones a modelo
empleo_modelo.add_cpds(dpc_exp, dpc_pro, dpc_ent, dpc_adm, dpc_ofe)

# verificación del modelo
empleo_modelo.check_model()

True

### 2. Inferencia

In [9]:
# instanciando objecto de eliminación de variables
empleo_inf = VariableElimination(empleo_modelo)

# haciendo consulta P(E | O = 1)
consulta_exp_ofe = empleo_inf.query(variables=['Experiencia'], evidence={'Oferta': 1})
print(consulta_exp_ofe)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+----------------+--------------------+
| Experiencia    |   phi(Experiencia) |
+================+====================+
| Experiencia(0) |             0.5291 |
+----------------+--------------------+
| Experiencia(1) |             0.4709 |
+----------------+--------------------+
